In [2]:
import pickle
import os

In [3]:
DELTA_E_MODELS_DIR = os.path.join(
    os.path.dirname(os.path.dirname(os.getcwd())), "models/delta_E/"
)

SIM_TIME_MODELS_DIR = os.path.join(
    os.path.dirname(os.path.dirname(os.getcwd())), "models/sim_time/"
)

# Loading the models

In [ ]:
delta_E_model_name = 'random_forest_model.pkl'
sim_time_model_name